In [1]:
!pip install transformers torch

In [2]:
# 2. Подключение Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# 3. Импорт необходимых модулей
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
# 4. Путь к модели на Google Drive
model_path = "/content/drive/MyDrive/gemmaX2/"

# 5. Загрузка токенизатора и модели
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,  # Используем 16-bit точность для экономии памяти
    device_map="auto"           # Автоматически распределяем модель по доступным устройствам
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
# 6. Переводим модель в режим оценки
model.eval()

def generate_response(prompt, max_new_tokens=256):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            do_sample=True,
            repetition_penalty=1.2,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [17]:
# system_prompt = """
# Ты - система информационного поиска НИУ ВШЭ.
# Создай краткий ответ на основе предоставленных документов.
# Используй только информацию из источников.
# Отвечай на русском языке, не повторяйся.
# """

# question = "Как получить бесплатную психологическую помощь в НИУ ВШЭ"
# contexst = """Психологическая помощь и телефоны горячих линий\n
# Горячая линия для студентов и аспирантов НИУ ВШЭ работает круглосуточно.
# По телефону 8-800-250-04-33 или через чат можно получить консультацию психолога."""

# user_query = f"""
# {question}
# =========
# {contexst}
# """

# full_prompt = f"{system_prompt}\nВопрос: {user_query}\nОтвет:"




# response = generate_response(full_prompt)
# print(response.split("Ответ:")[-1].strip())

Чтобы воспользоваться услугой "Бесплатная психологическая помощь", необходимо иметь действующий логин пользователя и пароль от учебного кабинета НИУ ВШЭ. Также важно помнить адрес телефона горячей линии заведения, который указан на официальном сайте (https://vse.shhs.ru/content/hotline)


In [18]:
def generate_hse_answer(question: str, context: str, max_length: int = 512, temperature: float = 0.5) -> str:
    """
    Генерирует ответ на вопрос с использованием системного промпта НИУ ВШЭ

    Параметры:
    - question: вопрос пользователя
    - context: релевантные отрывки контекста
    - max_length: максимальная длина ответа
    - temperature: параметр креативности (0.1-1.0)

    Возвращает:
    - Очищенный FINAL ANSWER
    """
    # Системный промпт
    SYSTEM_PROMPT = f"""Ты - система информационного поиска НИУ ВШЭ. Отвечай только по предоставленному контексту.
Создай краткий ответ со ссылками на источники в формате:
QUESTION: {question}
=========
{context}
=========
FINAL ANSWER:"""

    # Формируем полный промпт
    full_prompt = SYSTEM_PROMPT.format(question=question, context=context)

    # Токенизация и генерация
    inputs = tokenizer(
        full_prompt,
        return_tensors="pt",
        max_length=2048,
        truncation=True
    ).to(model.device)

    # Параметры генерации
    generation_params = {
        "input_ids": inputs.input_ids,
        "attention_mask": inputs.attention_mask,
        "max_new_tokens": max_length,
        "temperature": temperature,
        "top_p": 0.9,
        "repetition_penalty": 1.2,
        "do_sample": True,
        "pad_token_id": tokenizer.eos_token_id,
        "eos_token_id": tokenizer.eos_token_id,
        "early_stopping": True
    }

    # Генерация
    with torch.no_grad():
        outputs = model.generate(**generation_params)

    # Декодирование и постобработка
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Извлечение FINAL ANSWER
    if "FINAL ANSWER:" in full_response:
        response = full_response.split("FINAL ANSWER:")[-1].strip()
        response = response.split("=========")[0].strip()
        response = response.split("</s>")[0].strip()

        # Удаление повторяющихся предложений
        sentences = []
        for sent in response.split('. '):
            if sent not in sentences:
                sentences.append(sent)
        return '. '.join(sentences)

    return "Ответ не найден в предоставленных материалах"

In [19]:
context = """Психологическая помощь и телефоны горячих линий\n
Горячая линия для студентов и аспирантов НИУ ВШЭ работает круглосуточно.
По телефону 8-800-250-04-33 или через чат можно получить консультацию психолога."""
question = "Как я могу получить бесплатную консультацию у психолога ВШЭ?"

answer = generate_hse_answer(
    question=question,
    context=context,
    temperature=0.3,
    max_length=300
)

print(f"Вопрос: {question}\nОтвет: {answer}")

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Вопрос: Как я могу получить бесплатную консультацию у психолога ВШЭ?
Ответ: Чтобы получить бесплатную консультацию от психолога ВШЭ, обратитесь к следующему адресу: https://psi.hse.ru/contact/#feedback_form
